In [1]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain
from langchain_core.runnables.base import RunnableSequence
import openai,pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

In [2]:
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model="gpt-4o-mini",api_key=api_key, temperature=0.3)

In [3]:
prompt_template = """
Given the following stock data for the past twelve months, predict the stock needs for the next month for each product:

{data}

The prediction should consider sales trends, product popularity, and stock shortages.

Additionally, provide recommendations to move products from stores with lower demand to stores with higher demand.

Provide predictions and recommendations in the following format:

- StoreID: 1, Product: Product A, Predicted Stock: <predicted_value>
- StoreID: 2, Product: Product B, Predicted Stock: <predicted_value>

Recommendations:
- Move <quantity> of Product A from Store <low_demand_store_id> to Store <high_demand_store_id>
"""

prompt = PromptTemplate(input_variables=["data"], template=prompt_template)


In [4]:
try:
    load_dotenv()
    db_username = os.getenv("db_username")
    db_password = os.getenv("db_password")
    db_host = os.getenv("db_hostname")
    db_name = os.getenv("db_database")
    db_server = os.getenv("db_server")
    print(db_username, db_password, db_name, db_server)

    connection_string = f'mssql+pyodbc://{db_username}:{db_password}@{db_server}/{db_name}?driver=ODBC+Driver+17+for+SQL+Server'
    engine = create_engine(connection_string)
except Exception as e:
    print("Error connecting to SQL Server:", e)
    sys.exit(1)


sa clouderp123! gear 103.172.151.143


In [5]:
query = """WITH StoreInOut AS (
    SELECT 
        ST.StoreID,
		ST.StoreName,
		P.ProdNum,
        P.ProdID,
        SUM(CASE WHEN PST.StoreInward = 1 THEN PST.StoreQty ELSE 0 END) AS StoreQtyIn,
        SUM(CASE WHEN PST.StoreInward = 0 THEN PST.StoreQty ELSE 0 END) AS StoreQtyOut,
        M.MonthShortName,
        M.MonthIndex,
        FY.FinancialYear
    FROM 
        STORES ST
        INNER JOIN PRODUCTSTORETRANSACTION PST ON PST.StoreID = ST.StoreID
        INNER JOIN PRODUCTS P ON P.ProdID = PST.ProdID
        INNER JOIN MONTHS M ON M.MonthIndex = MONTH(PST.ProdStoreTransDate)
        INNER JOIN FINANCIALYEAR FY ON FY.FinancialYear = YEAR(PST.ProdStoreTransDate)
	WHERE FY.FinancialYear >= YEAR(GETDATE()) - 1

    GROUP BY 
        ST.StoreID,
		ST.StoreName,
		P.ProdNum,
        P.ProdID,
        M.MonthShortName,
        M.MonthIndex,
        FY.FinancialYear
)

SELECT 
    SI.StoreName,
    SI.StoreID,
    SI.ProdNum,
    SI.ProdID,
    SI.StoreQtyIn AS StoreQty,
    ISNULL(SO.StoreQtyOut, 0) AS StoreQtyOut,
    SI.MonthShortName,
    SI.MonthIndex,
    SI.FinancialYear
FROM 
    StoreInOut SI
LEFT JOIN 
    StoreInOut SO ON SO.StoreID = SI.StoreID
                 AND SO.ProdID = SI.ProdID
                 AND SO.MonthIndex = SI.MonthIndex
                 AND SO.FinancialYear = SI.FinancialYear
ORDER BY 
    SI.FinancialYear, SI.MonthIndex ASC;
 
"""

In [6]:
df = pd.read_sql(query, engine)
engine.dispose()

print(df.columns)

Index(['StoreName', 'StoreID', 'ProdNum', 'ProdID', 'StoreQty', 'StoreQtyOut',
       'MonthShortName', 'MonthIndex', 'FinancialYear'],
      dtype='object')


In [7]:
df.head()

,StoreName,StoreID,ProdNum,ProdID,StoreQty,StoreQtyOut,MonthShortName,MonthIndex,FinancialYear
0,HO MFGS,6378554838671663091,ACCSUVEKT0509,6378587582580560261,2.0,1.0,Sep,9,2024
1,HO MFGS,6378554838671663091,BKPBONCE20112,6378587577507766181,2.0,1.0,Sep,9,2024
2,HO MFGS,6378554838671663091,BKPGALAXY0005,6378587674572045351,2.0,1.0,Sep,9,2024
3,HO MFGS,6378554838671663091,BKPINVENT1901,6381181355946377971,2.0,1.0,Sep,9,2024
4,HO MFGS,6378554838671663091,BKPSNDBRK1705,6381181218184537171,1.0,1.0,Sep,9,2024


In [8]:
df.drop(columns=['StoreName','ProdID', 'MonthIndex'], inplace=True)
print(df)

                 StoreID        ProdNum  StoreQty  StoreQtyOut MonthShortName  \
0    6378554838671663091  ACCSUVEKT0509       2.0          1.0            Sep   
1    6378554838671663091  BKPBONCE20112       2.0          1.0            Sep   
2    6378554838671663091  BKPGALAXY0005       2.0          1.0            Sep   
3    6378554838671663091  BKPINVENT1901       2.0          1.0            Sep   
4    6378554838671663091  BKPSNDBRK1705       1.0          1.0            Sep   
..                   ...            ...       ...          ...            ...   
927  6383712662411898861  RKSOUTDR20903       1.0          0.0            Dec   
928  6383712662411898861  RKSOUTLNR0512       0.0          1.0            Dec   
929  6383712662411898861  RKSOUTLNR0904       0.0          1.0            Dec   
930  6383712662411898861   TLPTURBO0113       0.0         10.0            Dec   
931  6383712662411898861  TLPTURBO40201       3.0          0.0            Dec   

     FinancialYear  
0     

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 932 entries, 0 to 931
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   StoreID         932 non-null    int64  
 1   ProdNum         932 non-null    object 
 2   StoreQty        932 non-null    float64
 3   StoreQtyOut     932 non-null    float64
 4   MonthShortName  932 non-null    object 
 5   FinancialYear   932 non-null    int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 43.8+ KB


In [10]:
df.shape

(932, 6)

In [11]:
df_string = df.to_string(index=False)

In [12]:
print(df_string)

            StoreID         ProdNum  StoreQty  StoreQtyOut MonthShortName  FinancialYear
6378554838671663091   ACCSUVEKT0509       2.0          1.0            Sep           2024
6378554838671663091   BKPBONCE20112       2.0          1.0            Sep           2024
6378554838671663091   BKPGALAXY0005       2.0          1.0            Sep           2024
6378554838671663091   BKPINVENT1901       2.0          1.0            Sep           2024
6378554838671663091   BKPSNDBRK1705       1.0          1.0            Sep           2024
6378554838671663091   DUFPRO2000106       2.0          1.0            Sep           2024
6378554838671663091   LBPCLASIC0519      34.0         17.0            Sep           2024
6378554838671663091   LBPCLSLTH1202       2.0          0.0            Sep           2024
6378554838671663091   LBPJNRNCR0403       2.0          1.0            Sep           2024
6378554838671663091   LBPVNTAG20501       2.0          1.0            Sep           2024
6379488495287461871  

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt

# summary = df.groupby(['store_id', 'product_id']).sum()

# # Display the summary
# # print(summary)

# # Plotting the stock vs. sales for each product in each store
# fig, ax = plt.subplots()

# for store_id in df['store_id'].unique():
#     for product in df[df['store_id'] == store_id]['product_id'].unique():
#         data = df[(df['store_id'] == store_id) & (df['product_id'] == product)]
#         ax.plot(data['date'], data['stock'], label=f'Store {store_id} - Product {product} Stock')
#         ax.plot(data['date'], data['sales'], label=f'Store {store_id} - Product {product} Sales')

# ax.set_xlabel('date')
# ax.set_ylabel('Count')
# ax.set_title('Stock vs. Sales Comparison')
# ax.legend(loc='best')
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()


In [14]:
sequence = RunnableSequence(prompt, llm)

prediction = sequence.invoke({"data": df_string})

print(prediction.content)

Based on the provided stock data, I will analyze the sales trends, product popularity, and stock shortages to predict the stock needs for the next month (December 2024) for each product. Additionally, I will provide recommendations for moving products from stores with lower demand to those with higher demand.

### Predictions:

- StoreID: 6378554838671663091, Product: ACCSUVEKT0509, Predicted Stock: 50
- StoreID: 6378554838671663091, Product: BKPBONCE20112, Predicted Stock: 625
- StoreID: 6378554838671663091, Product: BKPGALAXY0005, Predicted Stock: 100
- StoreID: 6378554838671663091, Product: BKPINVENT1901, Predicted Stock: 5
- StoreID: 6378554838671663091, Product: BKPSNDBRK1705, Predicted Stock: 100
- StoreID: 6378554838671663091, Product: DUFPRO2000106, Predicted Stock: 3
- StoreID: 6378554838671663091, Product: LBPCLASIC0519, Predicted Stock: 50
- StoreID: 6378554838671663091, Product: LBPCLSLTH1202, Predicted Stock: 50
- StoreID: 6378554838671663091, Product: LBPJNRNCR0403, Predi

In [15]:
import markdown

html = markdown.markdown(prediction.content)

with open('stock_vs_sales_using_OpenAI.html', 'w') as file:
    file.write(html)

print("HTML content saved to stock_vs_sales.html")


HTML content saved to stock_vs_sales.html
